In [2]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
#import inflect #numeric/cardinal

In [3]:
dirPath = r"/data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"

path = dirPath + "/" + reviewPath

In [4]:
import os
current = os.getcwd()
current
path = current + path
path

'/home/tone/Study/NLP/data/reviews_Tools_and_Home_Improvement_5.json.gz'

In [5]:
dataFile = gzip.open(path, 'r')

In [6]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [7]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [8]:
len(productReviews)

134476

In [8]:
#productReviews[101718]

In [9]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [10]:
productsReviewData.shape
productsReviewData.columns
productsReviewData.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A4IL0CLL27Q33,104800001X,D. Brennan,"[0, 1]","I hate it when my shirt collars, not otherwise...",5.0,Perfect for collar stay management,1390953600,"01 29, 2014"
1,A3Q5W5E7TDVLJF,104800001X,funnyc130,"[0, 0]",These little magnets are really powerful for t...,5.0,Neat,1369958400,"05 31, 2013"


In [11]:
reduceData = productsReviewData.loc[:, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]


In [12]:
reduceData = reduceData.loc[reduceData['overall']!=3.0, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]

In [13]:
rating = lambda x: 1 if x > 2.0 else 0

In [14]:
reduceData['rating'] = reduceData['overall'].apply(rating)
unfavorableReviews = list(reduceData.loc[ reduceData['rating'] == 0, 
                                         ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').index)
#unfavorableReviews
reduceData.loc[ reduceData['rating'] == 0, ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').head(4)


,reviewText,helpful,overall,rating
134322,Amazing that they designed this thing without ...,"[2, 2]",1.0,0
55236,I was ready to pick up one of these at a woodw...,"[30, 38]",1.0,0
55234,I bought this set from amazon but is not like ...,"[8, 15]",1.0,0
105852,Horrible.Not one blade runs true. Some are as...,"[0, 0]",1.0,0


In [15]:
reviewTextProcessing = pd.DataFrame(reduceData.reviewText.copy())

In [16]:
reviewTextProcessing['rating'] = reduceData['rating']
reviewTextProcessing['overall'] = reduceData['overall']

In [17]:
productsReviewData.dtypes
len(productsReviewData)
# reviewText, rating, overall
reviewTextProcessing.shape, reviewTextProcessing.columns

((123707, 3), Index(['reviewText', 'rating', 'overall'], dtype='object'))

In [18]:
#reviewTextProcessing.loc[reviewTextProcessing['rating']==0, ['rating', 'overall', 'Processed', 'SentimentModel']]

In [19]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

#print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)


In [20]:
sample1 = "s***"
print(re.sub("\*", "#", sample1))

sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)
expand_contractions(sample1)

s###


'shit'

In [21]:
#from ntlk import tokenize
import nltk

In [27]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        review[count] = " ".join(eachSent)
    wholeText = [" ".join(review) ]
    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

#paragraphText, wholeText = singleReviewCleanUp(sample)
#wholeText, paragraphText
#paragraphText

In [28]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [29]:
#%%time
reviewTextProcessing['Processed'] = reviewTextProcessing.reviewText.apply(cleanUpTextParagraph)


In [30]:
reviewTextProcessing['CleanedJoin'] = reviewTextProcessing['Processed'].apply(', '.join)

In [31]:
reviewTextProcessing.shape
reviewTextProcessing.columns

Index(['reviewText', 'rating', 'overall', 'Processed', 'CleanedJoin'], dtype='object')

In [32]:
reviewTextProcessing.sort_values(by='rating', inplace=True)

In [33]:
reviewTextProcessing.head(5)

,reviewText,rating,overall,Processed,CleanedJoin
99410,Bought one and loved it! Ordered another two.O...,0,1.0,"[bought one and loved it, ordered another two ...","bought one and loved it, ordered another two o..."
79663,What a waste of money. Brand new and worse th...,0,1.0,"[what a waste of money, brand new and worse th...","what a waste of money, brand new and worse tha..."
79664,I just learned that there is another battery b...,0,1.0,[i just learned that there is another battery ...,i just learned that there is another battery b...
123031,I needed to cut 1/4&#34; off the edge of a clo...,0,1.0,[i needed to cut off the edge of a closet door...,i needed to cut off the edge of a closet door ...
12262,Used this for a while. Handy if it's all you ...,0,2.0,"[used this for a while, handy if it has it is ...","used this for a while, handy if it has it is a..."


In [34]:
size = reviewTextProcessing.loc[reviewTextProcessing['rating'] == 0, :].shape[0]
size

10105

In [35]:
reviewTextProcessing = reviewTextProcessing.iloc[0:size*5, :]


In [36]:
additionalNegatives = reviewTextProcessing.iloc[0:size, :].copy()
additionalNegatives.head(2)

,reviewText,rating,overall,Processed,CleanedJoin
99410,Bought one and loved it! Ordered another two.O...,0,1.0,"[bought one and loved it, ordered another two ...","bought one and loved it, ordered another two o..."
79663,What a waste of money. Brand new and worse th...,0,1.0,"[what a waste of money, brand new and worse th...","what a waste of money, brand new and worse tha..."


In [69]:
testSample = pd.concat([reviewTextProcessing, additionalNegatives])
#testSample = reviewTextProcessing

In [70]:
testSample = testSample.reset_index()
#testSample = testSample.drop_duplicates(subset="index")


In [71]:
testSample.shape

(60630, 6)

In [72]:
import itertools

largeText = testSample.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

60630

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
#import spacy


In [73]:
X = largeText
y = testSample['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=310)

x_train_text = X_train
x_test_text = X_test


In [74]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(45472, 15158, 45472, 15158)

In [75]:
from sklearn.naive_bayes import MultinomialNB

In [39]:
#import tensorflow as tf
#tf.__version__

'1.14.0'

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
top_num_of_words = 15000
#vect = TfidfVectorizer(max_features=top_num_of_words, stop_words = 'english')
vect = TfidfVectorizer(max_features=top_num_of_words, stop_words = 'english', ngram_range=(1, 3), max_df=0.9, min_df=2, sublinear_tf=True)
#vect = TfidfVectorizer(max_features=top_num_of_words, ngram_range=(1, 2), max_df=0.9, min_df=2, use_idf=True, sublinear_tf=True)


In [41]:
#x_train_text

In [77]:
text = x_train_text
#text[0]

In [78]:
vect.fit(text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=15000, min_df=2,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [79]:
tfidf_train = vect.transform(text)
tfidf_train

<45472x15000 sparse matrix of type '<class 'numpy.float64'>'
	with 2160120 stored elements in Compressed Sparse Row format>

In [80]:
model = MultinomialNB()

In [46]:
#model = LogisticRegression(C=0.2, dual=True)
#(0, 2418, 0, 31201) nooo negative

In [81]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)

In [82]:
index = len(x_train_text)
index

45472

In [83]:
model.fit(tfidf_train, y_train)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [84]:
vect.fit(x_test_text)
tfidf_test = vect.transform(x_test_text)

In [85]:
tfidf_test

<15158x15000 sparse matrix of type '<class 'numpy.float64'>'
	with 733613 stored elements in Compressed Sparse Row format>

In [86]:
preds = model.predict(tfidf_test)
accuracy = (preds==y_test).mean()


In [87]:
print("Accuracy: {0:.2%}".format(accuracy))


Accuracy: 65.67%


In [88]:
from sklearn.metrics import confusion_matrix

In [89]:
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
tn, fp, fn, tp

(1296, 3762, 1442, 8658)

In [90]:
len(y_test)

15158

In [56]:
pd.options.display.max_colwidth = 500

#productsReviewData.cleanedText.head(2)